In [1]:
import pandas as pd

In [32]:
transaction = pd.read_excel('ERP data 02.12.2024.xlsx')

In [33]:
transaction.head()

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
0,1.000167e+09,1.849004e+09,2018-08-06,2018-08-02 00:00:00,SI,DO/2000-BGMF,DO/2000-BGMF001,2018.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
1,1.000167e+09,1.849004e+09,2018-08-06,2018-08-02 00:00:00,SI,DO/2000-BGMF,DO/2000-BGMF001,2018.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
2,1.001850e+09,2.349008e+09,2023-09-27,2023-09-27 00:00:00,SI,DO/2000-BGMF,DO/2000-BGMF001,2023.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
3,1.001850e+09,2.349008e+09,2023-09-27,2023-09-27 00:00:00,SI,DO/2000-BGMF,DO/2000-BGMF001,2023.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
4,1.002169e+09,2.449006e+09,2024-09-03,2024-09-03 00:00:00,SI,DO/2000-BGMF,DO/2000-BGMF001,2024.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN


In [34]:
wbs = pd.read_excel('Data.xlsx', sheet_name='WBS')

In [35]:
transaction.to_csv('Transaction.csv', quoting=1, lineterminator='\r\n', index=False)

In [36]:
wbs.to_csv('WBS.csv', quoting=1, lineterminator='\r\n', index=False)

In [37]:
transaction[transaction['Document Date'] == '1019-10-21']

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
71250,1.000589e+09,1.917021e+09,2019-11-18,1019-10-21,KR,DO/2016-IDEAS,DO/2016-IDEAS01,2019.0,510076.0,EXP-OTHER CHARGES,...,M,4.0,13282.5,INR,13282.5,13282.5,13282.5,13282.5,13282.5,NaN
71251,1.000589e+09,1.917021e+09,2019-11-18,1019-10-21,KR,DO/2016-IDEAS,DO/2016-IDEAS01,2019.0,510076.0,EXP-OTHER CHARGES,...,M,4.0,1575.0,INR,1575.0,1575.0,1575.0,1575.0,1575.0,NaN
98759,1.000589e+09,1.917021e+09,2019-11-18,1019-10-21,KR,DO/2019-EDPG,DO/2019-EDPG001,2019.0,510076.0,EXP-OTHER CHARGES,...,M,4.0,20737.5,INR,20737.5,20737.5,20737.5,20737.5,20737.5,NaN


## Convert Document date to datetime format

In [38]:
# Assuming 'transaction' is your dataframe
transaction['Document Date'] = pd.to_datetime(transaction['Document Date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Replace invalid dates with values from the 'Posting Date' column
transaction['Document Date'] = transaction['Document Date'].fillna(transaction['Posting Date'])

## Drop blank columns

In [39]:
transaction = transaction.dropna(subset=['Document type']).reset_index(drop=True)

## Perform WBS mapping

In [67]:
transaction = transaction.merge(wbs[['WBS ', 'WBS Details ']].drop_duplicates(), how='left', left_on='Project definition', right_on='WBS ')

In [73]:
transaction = transaction.merge(wbs[['Sub WBS ', 'SUB WBS Details']].drop_duplicates(), how='left', left_on='Object', right_on='Sub WBS ')

In [72]:
wbs.columns

Index(['WBS ', 'Sub WBS ', 'SUB WBS Details', 'WBS Details ', 'PI Name'], dtype='object')

## Get non-HF Donations

In [41]:
transaction[transaction['Name of offsetting account'].str.contains('IITBHF')]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
427,1.001066e+09,2.126001e+09,2021-05-27,2021-05-27,SB,DO/2000-POOL,DO/2000-POOL001,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,230569.34,INR,230569.34,230569.34,230569.34,230569.34,230569.34,NaN
1642,1.000905e+09,2.026004e+09,2020-11-20,2020-11-20,SB,DO/2000-POOL,DO/2000-POOL002,2020.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-19920841.77,INR,-19920841.77,-19920841.77,-19920841.77,-19920841.77,-19920841.77,NaN
1643,1.001066e+09,2.126001e+09,2021-05-27,2021-05-27,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,251000.77,INR,251000.77,251000.77,251000.77,251000.77,251000.77,NaN
1644,1.001095e+09,2.126002e+09,2021-08-03,2021-08-03,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-22861903.85,INR,-22861903.85,-22861903.85,-22861903.85,-22861903.85,-22861903.85,NaN
1645,1.001235e+09,2.126007e+09,2022-01-13,2022-01-13,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-257402.03,INR,-257402.03,-257402.03,-257402.03,-257402.03,-257402.03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133942,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VBAR,DO/2024-VBARREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-208457.00,INR,-208457.00,-208457.00,-208457.00,-208457.00,-208457.00,NaN
133948,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-3126858.00,INR,-3126858.00,-3126858.00,-3126858.00,-3126858.00,-3126858.00,NaN
133949,1.002124e+09,2.426003e+09,2024-07-12,2024-07-05,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-1250040.00,INR,-1250040.00,-1250040.00,-1250040.00,-1250040.00,-1250040.00,NaN
133955,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-3126862.00,INR,-3126862.00,-3126862.00,-3126862.00,-3126862.00,-3126862.00,NaN


In [42]:
transaction[transaction['Offsetting acct no.'] == '550511']

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
427,1.001066e+09,2.126001e+09,2021-05-27,2021-05-27,SB,DO/2000-POOL,DO/2000-POOL001,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,230569.34,INR,230569.34,230569.34,230569.34,230569.34,230569.34,NaN
1642,1.000905e+09,2.026004e+09,2020-11-20,2020-11-20,SB,DO/2000-POOL,DO/2000-POOL002,2020.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-19920841.77,INR,-19920841.77,-19920841.77,-19920841.77,-19920841.77,-19920841.77,NaN
1643,1.001066e+09,2.126001e+09,2021-05-27,2021-05-27,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,251000.77,INR,251000.77,251000.77,251000.77,251000.77,251000.77,NaN
1644,1.001095e+09,2.126002e+09,2021-08-03,2021-08-03,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-22861903.85,INR,-22861903.85,-22861903.85,-22861903.85,-22861903.85,-22861903.85,NaN
1645,1.001235e+09,2.126007e+09,2022-01-13,2022-01-13,SB,DO/2000-POOL,DO/2000-POOL002,2021.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-257402.03,INR,-257402.03,-257402.03,-257402.03,-257402.03,-257402.03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133942,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VBAR,DO/2024-VBARREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-208457.00,INR,-208457.00,-208457.00,-208457.00,-208457.00,-208457.00,NaN
133948,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-3126858.00,INR,-3126858.00,-3126858.00,-3126858.00,-3126858.00,-3126858.00,NaN
133949,1.002124e+09,2.426003e+09,2024-07-12,2024-07-05,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-1250040.00,INR,-1250040.00,-1250040.00,-1250040.00,-1250040.00,-1250040.00,NaN
133955,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-3126862.00,INR,-3126862.00,-3126862.00,-3126862.00,-3126862.00,-3126862.00,NaN


In [48]:
start_date = pd.to_datetime('01-04-2024', format='%d-%m-%Y')
end_date = pd.to_datetime('05-12-2024', format='%d-%m-%Y')

In [50]:
start_date

Timestamp('2024-04-01 00:00:00')

In [56]:
indian_donations = transaction[
    (transaction['Document type'] == 'DR') &
    (transaction['Document Date'].between(start_date, end_date)) &
    (
        ~(transaction['Name of offsetting account'].str.contains('IITBHF')) |
        ~(transaction['Cost Element'].isin([550510, 550511]))
    )
]

In [57]:
indian_donations

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
165,1.002047e+09,2.406000e+09,2024-04-15,2024-04-02,DR,DO/2000-FDFE,DO/2000-FDFEREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-2000.0,INR,-2000.0,-2000.0,-2000.0,-2000.0,-2000.0,NaN
166,1.002068e+09,2.406001e+09,2024-05-02,2024-05-02,DR,DO/2000-FDFE,DO/2000-FDFEREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-2000.0,INR,-2000.0,-2000.0,-2000.0,-2000.0,-2000.0,NaN
167,1.002089e+09,2.406002e+09,2024-06-03,2024-06-01,DR,DO/2000-FDFE,DO/2000-FDFEREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-2000.0,INR,-2000.0,-2000.0,-2000.0,-2000.0,-2000.0,NaN
168,1.002105e+09,2.406003e+09,2024-06-26,2024-06-24,DR,DO/2000-FDFE,DO/2000-FDFEREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-29.0,INR,-29.0,-29.0,-29.0,-29.0,-29.0,NaN
169,1.002109e+09,2.406003e+09,2024-07-02,2024-07-01,DR,DO/2000-FDFE,DO/2000-FDFEREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-2000.0,INR,-2000.0,-2000.0,-2000.0,-2000.0,-2000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133925,1.002179e+09,2.406010e+09,2024-09-13,2024-09-12,DR,DO/2024-RXOS,DO/2024-RXOSREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-1004652.0,INR,-1004652.0,-1004652.0,-1004652.0,-1004652.0,-1004652.0,NaN
133926,1.002063e+09,2.406001e+09,2024-04-22,2024-04-18,DR,DO/2024-SAHA,DO/2024-SAHAREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-20000.0,INR,-20000.0,-20000.0,-20000.0,-20000.0,-20000.0,NaN
133927,1.002063e+09,2.406001e+09,2024-04-22,2024-04-21,DR,DO/2024-SAHA,DO/2024-SAHAREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-10000.0,INR,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,NaN
133928,1.002077e+09,2.406002e+09,2024-05-09,2024-05-06,DR,DO/2024-SAHA,DO/2024-SAHAREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-5000.0,INR,-5000.0,-5000.0,-5000.0,-5000.0,-5000.0,NaN


In [76]:
non_indian_hf_donations = transaction[
    (transaction['Document type'] == 'DR') &
    (transaction['Document Date'].between(start_date, end_date)) &
    (transaction['Cost Element'] == 550510)
]

In [77]:
non_indian_hf_donations

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details
9739,1.002228e+09,2.406012e+09,2024-11-05,2024-10-25,DR,DO/2007-YFA,DO/2007-YFAREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-2996367.00,-2996367.00,-2996367.00,-2996367.00,-2996367.00,NaN,DO/2007-YFA,CLASS OF'82 LEGACY PROJECT JOINING BONUS,DO/2007-YFAREV,CLASS OF'82 LEGACY PROJECT -REV
77584,1.002138e+09,2.406006e+09,2024-07-25,2024-07-23,DR,DO/2018-CL98,DO/2018-CL98REV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-1488501.00,-1488501.00,-1488501.00,-1488501.00,-1488501.00,NaN,DO/2018-CL98,CLASS OF 1998-GIFT,DO/2018-CL98REV,CLASS OF 1998-GIFT
133233,1.002209e+09,2.406011e+09,2024-10-22,2024-10-22,DR,DO/2023-SYNO,DO/2023-SYNOREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-15492857.30,-15492857.30,-15492857.30,-15492857.30,-15492857.30,NaN,DO/2023-SYNO,Synopsys Semiconductor lab for Virtual F,DO/2023-SYNOREV,Synopsys Semicond. lab for Virtual-REV
133744,1.002138e+09,2.406006e+09,2024-07-25,2024-07-09,DR,DO/2024-ESRT,DO/2024-ESRTREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-666814.24,-666814.24,-666814.24,-666814.24,-666814.24,NaN,DO/2024-ESRT,ESR Technology Limited - Research Activi,DO/2024-ESRTREV,ESR Technology Limited - Research-REV
133745,1.002182e+09,2.406010e+09,2024-09-20,2024-09-13,DR,DO/2024-ESRT,DO/2024-ESRTREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-687410.70,-687410.70,-687410.70,-687410.70,-687410.70,NaN,DO/2024-ESRT,ESR Technology Limited - Research Activi,DO/2024-ESRTREV,ESR Technology Limited - Research-REV
133857,1.002068e+09,2.406001e+09,2024-05-01,2024-04-26,DR,DO/2024-INCO,DO/2024-INCOREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-4152470.00,-4152470.00,-4152470.00,-4152470.00,-4152470.00,NaN,DO/2024-INCO,IN COVID SUPPORT FZE LLC BFI BIOME,DO/2024-INCOREV,IN COVID SUPPORT FZE LLC BFI BIOME
133858,1.002068e+09,2.406001e+09,2024-05-01,2024-04-26,DR,DO/2024-INCO,DO/2024-INCOREV,2024.0,550510.0,Rev-Other Donation (Except IITBHF),...,-20787970.00,-20787970.00,-20787970.00,-20787970.00,-20787970.00,NaN,DO/2024-INCO,IN COVID SUPPORT FZE LLC BFI BIOME,DO/2024-INCOREV,IN COVID SUPPORT FZE LLC BFI BIOME


## Raised in US

In [78]:
hf_donations_raised = transaction[
    (transaction['Document type'] == 'DR') &
    (transaction['Document Date'].between(start_date, end_date)) &
    (transaction['Cost Element'] == 550511)
]

In [79]:
hf_donations_raised

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details
100573,1.002053e+09,2.406001e+09,2024-04-17,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-8.338300e+06,-8.338300e+06,-8.338300e+06,-8.338300e+06,-8.338300e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100575,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-1.250745e+07,-1.250745e+07,-1.250745e+07,-1.250745e+07,-1.250745e+07,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100576,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-3.126862e+06,-3.126862e+06,-3.126862e+06,-3.126862e+06,-3.126862e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100577,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-3.126858e+06,-3.126858e+06,-3.126858e+06,-3.126858e+06,-3.126858e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100578,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-3.039310e+05,-3.039310e+05,-3.039310e+05,-3.039310e+05,-3.039310e+05,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100579,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,3.039310e+05,3.039310e+05,3.039310e+05,3.039310e+05,3.039310e+05,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100580,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-1.919357e+06,-1.919357e+06,-1.919357e+06,-1.919357e+06,-1.919357e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100581,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-2.167960e+05,-2.167960e+05,-2.167960e+05,-2.167960e+05,-2.167960e+05,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100582,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-2.584869e+06,-2.584869e+06,-2.584869e+06,-2.584869e+06,-2.584869e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256
100583,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-1.021024e+06,-1.021024e+06,-1.021024e+06,-1.021024e+06,-1.021024e+06,NaN,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256


### Allocated

In [80]:
hf_donations_allocated = transaction[
    (transaction['Document type'] == 'SB') &
    (transaction['Document Date'].between(start_date, end_date)) &
    (transaction['Cost Element'] == 550511)
]

In [81]:
hf_donations_allocated

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details
1900,1.002200e+09,2.426007e+09,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,-32196387.45,-32196387.45,NaN,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP
1901,1.002200e+09,2.426007e+09,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,32196387.45,32196387.45,32196387.45,32196387.45,32196387.45,NaN,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP
2717,1.002200e+09,2.426007e+09,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2024-POOL002-004,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,-32196387.45,-32196387.45,NaN,DO/2000-POOL,Donation Pool Account,DO/2024-POOL002-004,IITBHF GRANT #259
5955,1.002124e+09,2.426003e+09,2024-07-12,2024-07-05,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-6250275.00,-6250275.00,-6250275.00,-6250275.00,-6250275.00,NaN,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA
5956,1.002201e+09,2.426007e+09,2024-10-14,2024-10-14,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-16733800.00,-16733800.00,-16733800.00,-16733800.00,-16733800.00,NaN,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133942,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VBAR,DO/2024-VBARREV,2024.0,550511.0,Rev-IITBHF Donation,...,-208457.00,-208457.00,-208457.00,-208457.00,-208457.00,NaN,DO/2024-VBAR,Dr. Varun Bhalerao's Astronomy Related A,DO/2024-VBARREV,Dr. Varun Bhalerao's Astronomy Rela-REV
133948,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126858.00,-3126858.00,-3126858.00,-3126858.00,-3126858.00,NaN,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV
133949,1.002124e+09,2.426003e+09,2024-07-12,2024-07-05,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-1250040.00,-1250040.00,-1250040.00,-1250040.00,-1250040.00,NaN,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV
133955,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126862.00,-3126862.00,-3126862.00,-3126862.00,-3126862.00,NaN,DO/2024-VCSF,Vijay Vashee Computer Science Fellowship,DO/2024-VCSFREV,Vijay Vashee Computer Science Fell-REV


In [83]:
abs(hf_donations_allocated['Val/COArea Crcy'].sum())

np.float64(2.9802322387695312e-08)

In [21]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133957 entries, 0 to 133956
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Document Number             133957 non-null  float64       
 1   Ref. document number        133957 non-null  float64       
 2   Posting Date                133957 non-null  datetime64[ns]
 3   Document Date               133957 non-null  datetime64[ns]
 4   Document type               133957 non-null  object        
 5   Project definition          133957 non-null  object        
 6   Object                      133957 non-null  object        
 7   Fiscal Year                 133957 non-null  float64       
 8   Cost Element                133957 non-null  float64       
 9   Cost element descr.         133957 non-null  object        
 10  Dr/Cr indicator             133957 non-null  object        
 11  Val/COArea Crcy             133957 non-

In [22]:
transaction

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Exchange Rate Type,Value Type,Value in Obj. Crcy,Report currency,Val.in rep.cur.,Vbl.Val./COCrcy,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text
0,1.000167e+09,1.849004e+09,2018-08-06,2018-08-02,SI,DO/2000-BGMF,DO/2000-BGMF001,2018.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
1,1.000167e+09,1.849004e+09,2018-08-06,2018-08-02,SI,DO/2000-BGMF,DO/2000-BGMF001,2018.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
2,1.001850e+09,2.349008e+09,2023-09-27,2023-09-27,SI,DO/2000-BGMF,DO/2000-BGMF001,2023.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
3,1.001850e+09,2.349008e+09,2023-09-27,2023-09-27,SI,DO/2000-BGMF,DO/2000-BGMF001,2023.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
4,1.002169e+09,2.449006e+09,2024-09-03,2024-09-03,SI,DO/2000-BGMF,DO/2000-BGMF001,2024.0,510162.0,EXP-Awards,...,M,4.0,10000.0,INR,10000.0,10000.0,10000.0,10000.0,10000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133952,1.002167e+09,2.426005e+09,2024-08-30,2024-08-30,SB,DO/2024-VCSF,DO/2024-VCSF001,2024.0,550054.0,REV-Awards,...,M,4.0,-150000.0,INR,-150000.0,-150000.0,-150000.0,-150000.0,-150000.0,NaN
133953,1.002167e+09,2.426005e+09,2024-08-30,2024-08-30,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,150000.0,INR,150000.0,150000.0,150000.0,150000.0,150000.0,NaN
133954,1.002184e+09,2.426006e+09,2024-09-24,2024-09-24,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,M,4.0,-2311350.0,INR,-2311350.0,-2311350.0,-2311350.0,-2311350.0,-2311350.0,NaN
133955,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,M,4.0,-3126862.0,INR,-3126862.0,-3126862.0,-3126862.0,-3126862.0,-3126862.0,NaN


In [20]:
project_directory = pd.read_excel('Financials_Q4 Purpose_Kamal_17_05_24 (1).xlsx', sheet_name='Directory', header=None)

In [21]:
project_directory.columns = ['project_name', 'category']

In [22]:
project_directory['project_id'] = project_directory['project_name'].str.strip().str.lower().str.replace(' ', '_')

In [24]:
project_directory = project_directory[['project_id', 'project_name', 'category']]

In [25]:
project_directory['category'].value_counts()

category
Others                             67
R&D                                44
Infrastructure                     19
COE                                14
Scholarship                        13
Centre                             11
Chair                              10
Labs                                8
Awards                              8
Evergreen                           7
Entreprenurship                     5
RFW                                 2
Research Hub                        2
Fellowship                          2
Faculty Awards                      2
FAP                                 2
Awards + EV lab                     1
Entreprenurship + Student teams     1
FDF                                 1
Infrastructure,R&D,Urgent needs     1
Legacy-TBD                          1
Lecture Series                      1
Scholarship + M Engg                1
Name: count, dtype: int64

In [26]:
project_directory[~project_directory['category'].isin(['Centre', 'COE', 'R&D', 'Strategic Needs', 'Infrastructure', 'Project Evergreen', 'Evergreen', 'Student Development', 'Faculty Development', 'Awards', 'Chair'])]

,project_id,project_name,category
8,class_of_1973_legacy_project,Class of 1973 Legacy Project,Awards + EV lab
44,class_of_1970_legacy_project,Class of 1970 Legacy Project,Entreprenurship
45,entrepreneurship_and_innovation,Entrepreneurship and Innovation,Entreprenurship
46,faculty_-_others,Faculty - Others,Entreprenurship
47,sine,SINE,Entreprenurship
...,...,...,...
218,scholarship_for_b.tech_students,Scholarship for B.Tech Students,Scholarship
219,scholarships_fellowships_and_entrepreneurship_...,Scholarships Fellowships and Entrepreneurship ...,Scholarship
220,student_scholarships,Student Scholarships,Scholarship
221,the_chemours_future_of_chemistry_scholarship_iitb,The Chemours Future of Chemistry Scholarship IITB,Scholarship


In [27]:
project_directory[project_directory['category'] == 'Centre']

,project_id,project_name,category
9,4th_annual_instalment_towards_operating_expenses,4th annual instalment towards operating expenses,Centre
10,centre_for_excellence_in_steel_technology,Centre for excellence in Steel Technology,Centre
11,centre_for_machine_intelligence_and_data_scien...,Centre for Machine Intelligence and Data Scien...,Centre
12,centre_of_excellence_in_oil_gas_and_energy_(co...,Centre of Excellence in Oil Gas and Energy (Co...,Centre
13,digital_trust_lab_at_the_department_of_compute...,Digital Trust Lab at the Department of Compute...,Centre
14,he-iitb_innovation_lab_at_iit_bombay,HE-IITB Innovation Lab at IIT Bombay,Centre
15,ongoing_research_activities_(year_3),Ongoing Research Activities (Year 3),Centre
16,ongoing_research_activities_(year_four),Ongoing Research Activities (Year FOUR),Centre
17,sbi_foundation_hub_for_data_&_analytics_for_india,SBI Foundation Hub for Data & Analytics for India,Centre
18,scan_project,SCAN project,Centre


In [23]:
project_directory

,project_name,category,project_id
0,DR.MANU AKULA MEMORIAL AWARD,Awards,dr.manu_akula_memorial_award
1,Kanitkar Merit Awards,Awards,kanitkar_merit_awards
2,Mainak Das Excellence in Publication Award,Awards,mainak_das_excellence_in_publication_award
3,Pamela and Shiv Khosla Merit Award,Awards,pamela_and_shiv_khosla_merit_award
4,Prashant Dave Best Ph.D.Thesis Award,Awards,prashant_dave_best_ph.d.thesis_award
...,...,...,...
218,Scholarship for B.Tech Students,Scholarship,scholarship_for_b.tech_students
219,Scholarships Fellowships and Entrepreneurship ...,Scholarship,scholarships_fellowships_and_entrepreneurship_...
220,Student Scholarships,Scholarship,student_scholarships
221,The Chemours Future of Chemistry Scholarship IITB,Scholarship,the_chemours_future_of_chemistry_scholarship_iitb


In [ ]:
project_directory.to_csv()